In [ ]:
import numpy as np
import importlib, os, datetime, pickle
from sus.protocol_designer import System, Protocol, Potential, Compound_Protocol
from sus.protocol_designer.protocol import sequential_protocol
from IPython import display
from IPython.display import HTML, Image
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation, PillowWriter
from quick_sim import setup_sim


# import edward_tools.fq_runner as fq_runner
from edward_tools.coupled_fq_potential import coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force, \
coupled_flux_qubit_non_linear_approx_pot_special, coupled_flux_qubit_non_linear_approx_force_special, coupled_flux_qubit_non_linear_approx_pot_break_down
from edward_tools.visualization import animate_sim_flux_qubit, plotFidelityBarChart, separate_by_state_2
from edward_tools.initial_state_sampling import extra_constraint_00_and_11_only
from NAND_PARAMETERS import *
import importlib

import kyle_tools as kt
import matplotlib.pyplot as plt


from edward_tools import coupled_fq_protocol_library, cfq_runner
from edward_tools import coupled_fq_protocol_library

import edward_tools.cfq_batch_sweep as cfq_batch_sweep
import edward_tools.Analysis_tool.general_analysis_tools as general_analysis_tool
# from edward_tools.Analysis_tool.general_analysis_tools import show_phi_dc_with_time
import edward_tools.Analysis_tool.minimum_value_of_potential as minimum_value_of_potential
from edward_tools.couple_flux_qubit_metrics import fidelityEvaluation
from edward_tools import visualization

coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_shot_at_different_t = coupled_fq_protocol_library.get_potential_shot_at_different_t
get_potential_shot_at_different_t_1D = coupled_fq_protocol_library.get_potential_shot_at_different_t_1D
create_simple_protocol_parameter_dict = coupled_fq_protocol_library.create_simple_protocol_parameter_dict
coupled_fq_runner = importlib.reload(cfq_runner)
coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_along_a_1D_cutline = coupled_fq_protocol_library.get_potential_along_a_1D_cutline
plotCutlines = coupled_fq_protocol_library.plotCutlines

# parameters

### circuit parameters

In [ ]:
circuit_params_from_Chris = {
    "L": 5e-12 * 1, "T":4.2, "L_factor": 1, 
    "beta": 2.3, "d_beta": 0,
    "phi_1x": 0.61,   "phi_2x": 0.10, "phi_1xdc": 0, "phi_2xdc": 1.79, "M_12": 0.06
}

experiment_circuit_params = {
    "L": 5e-12, "T": 4.2, "L_factor": 1,
    "beta": 2.3, "d_beta": 0,
    "phi_1x": 0.58,   "phi_2x": 0.095, "phi_1xdc": 0, "phi_2xdc": 1.96,
    "M_12": 0.055
}


circuit_params_chosen = circuit_params_from_Chris
experiment_comment = "Experiment 3 (2024-10-22): Failed KE extraction protocol simulation for Kyle's animation"

phi_1_x_on_12, phi_2_x_on_12, phi_1_dcx_on_12, phi_2_dcx_on_12, phi_1_x_on_21, phi_2_x_on_21, phi_1_dcx_on_21, phi_2_dcx_on_21, M_12_on, four_well, CE_1, CE_8, lower_H_wells, flip_H, lower_V_wells, flip_V, KE_extraction_protocol_for_CE_1, KE_extraction_protocol_for_CE_8 = generate_protocols_from_circuit_params(circuit_params_chosen)

In [ ]:
has_velocity = True

PHI_0 = 2.067833848 * 1e-15
k_B = 1.38e-23
T = circuit_params_chosen["T"]
k_BT = k_B * T

C_factor = 1
L_factor = circuit_params_chosen['L_factor']
R_factor = 30
I_m_factor = 0
time_scale = 1.0

I_p_1, I_p_2 = 5e-6 , 5e-6  # Amp
I_m_1, I_m_2 = 7e-9 * I_m_factor, 7e-9 * I_m_factor                           # Amp
R_1, R_2 = 1 * R_factor, 1 * R_factor                                         # ohm
C_1, C_2 = 500e-15 * C_factor, 500e-15 * C_factor                             # F

L_1, L_2 = 140e-12 * L_factor, 140e-12 * L_factor                             # H 
L_1, L_2 = 5e-12 * L_factor, 5e-12 * L_factor                             # H 
freq = 1/np.sqrt(C_1 * L_1)
characteristic_time = np.sqrt(C_1 * C_factor * L_1 * L_factor)

In [ ]:

m_c = C_1
m_1, m_2 = C_1, C_2
x_c = PHI_0 / (2 * np.pi)
time_scale_factor = 1
t_c = np.sqrt(L_1 * C_1)
v_c = x_c / t_c

U0_1, U0_2 = m_c * x_c**2 / t_c**2 / k_BT, m_2 * x_c**2 / t_c**2 / k_BT
kappa_1, kappa_2, kappa_3, kappa_4 = 1/U0_1, 1/U0_1, 1/U0_1, 1/U0_1

lambda_1 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_1  = 1
eta_1    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_1 / 1**2)

lambda_2 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_2  = 1 / (C_2/C_1)
eta_2    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_2 * (R_1 * C_1**2) / (R_2 * C_2**2))

lambda_3 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_3  = 4
eta_3    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_3)

lambda_4 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_4  = 4 / (C_2/C_1)
eta_4    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_4 * (R_1 * C_1**2) / (R_2 * C_2**2))

gamma = 10

beta_1 = 2.3
beta_2 = 2.3

d_beta_1 = 2 * np.pi * L_1 * I_m_1 / PHI_0; 
d_beta_2 = 2 * np.pi * L_2 * I_m_2 / PHI_0;

_lambda = np.array([lambda_1, lambda_2, lambda_3, lambda_4])
_theta  = np.array([theta_1, theta_2, theta_3, theta_4])
_eta  =   np.array([eta_1, eta_2, eta_3, eta_4])

v_1 = np.random.normal(0, np.sqrt(k_BT/m_1)) / v_c
v_2 = np.random.normal(0, np.sqrt(k_BT/m_2)) / v_c
v_3 = np.random.normal(0, np.sqrt(k_BT/(m_1/4))) / v_c
v_4 = np.random.normal(0, np.sqrt(k_BT/(m_2/4))) / v_c

In [ ]:
print(f"L_1 = {L_1 * 1e12:.3g}pH, T = {T}K")
print(f"freq = {freq / 1e9:.3g}GHz")
print(characteristic_time)
print(1/U0_1*50)

In [ ]:
k_BT / (x_c**2/L_1) * 50

### parameter setting

In [ ]:
"""
# step 0: modify parameters
- All the parameters are stored in a separate file PARAMETER_INPUT
- You can override some of the parameters here.
"""
params = {}
params['N'] = 1000
params['dt'] = 1/100
params['lambda'] = 1
params['beta'] = 1
params['sim_params'] = [_lambda, _theta, _eta]
params['target_work'] = None
params['applyOffset'] = False
params['measureWorkWithOffset'] = True
params['monitor_work_dist_in_whole_process'] = True # To monitor the work process
params['comment'] = experiment_comment
params['capacitance'] = np.array([C_1, C_2, C_1/4, C_2/4])
params['mass'] = np.array([1, 1, 1/4, 1/4])
params['v_c'] = x_c/t_c
params['k_BT'] = k_BT
params['U0'] = U0_1
params['as_step'] = np.s_[::] # the time step to skep for the all_state
params['percentage'] = 1 # For what percentage of the total sample do you want to keep in the output all_state


In [ ]:
"""
# step 2: Define initial condition and protocol
"""
manual_domain=[np.array([-5, -5]), np.array([5, 5])]



# µ = 0.06, φ2xdc = 1.79, φ1x = 0.59, and φ2x = 0.10.
    
initial_parameter_dict = {
        "U0_1": U0_1,     "U0_2": U0_2,     "gamma_1": gamma,  "gamma_2": gamma,
        "beta_1": beta_1,   "beta_2": beta_2,   "d_beta_1": d_beta_1 ,   "d_beta_2": d_beta_2,
        "phi_1_x": phi_1_x_off,  "phi_2_x": phi_2_x_off,  "phi_1_dcx": phi_1_dcx_off,  "phi_2_dcx": phi_1_dcx_off,
        "M_12": M_12_off, 'x_c': x_c
}


In [ ]:
"""
# step 1: Define potential
"""
coupled_fq_default_param = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, x_c]
[phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound] = np.array([4, 4, 4, 4])/time_scale_factor
contour_range = [300, 2000]
    
coupled_fq_domain = [[-phi_1_bound, -phi_2_bound, -phi_1dc_bound, -phi_2dc_bound], \
                     [phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound]]

# coupled_fq_pot = Potential(coupled_flux_qubit_pot_with_offset_at_00_xy, coupled_flux_qubit_force, 14, 4,\
#                            default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

coupled_fq_pot = Potential(coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force, 14, 4,\
                           default_params = initial_parameter_dict,  relevant_domain = coupled_fq_domain)

# coupled_fq_pot = Potential(coupled_flux_qubit_non_linear_approx_pot_special, coupled_flux_qubit_non_linear_approx_force_special, 14, 4,\
                           # default_params = initial_parameter_dict,  relevant_domain = coupled_fq_domain)

### protocol setting

In [ ]:
zeroDissipation = False
# zeroDissipation = True

saveAllStates = True

params['sim_params'] = [_lambda, _theta, _eta]

if zeroDissipation:
    params['sim_params'] = [_lambda * 0, _theta, _eta * 0]

params['circuit_parameters'] = {
    "C_factor":C_factor, "L_factor": L_factor, "R_factor": R_factor, "I_m_factor": I_m_factor, "T": T, 
    "I_p_1": I_p_1, "I_p_2": I_p_2, "I_m_1": I_m_1, "I_m_2": I_m_2,
    "R_1": R_1, "R_2": R_2, "C_1": C_1, "C_2": C_2, "L_1": L_1, "L_2": L_2, 
    "characteristic_time": np.sqrt(C_1 * C_factor * L_1 * L_factor),
    "phi_1_x_on": phi_1_x_on_12, "phi_2_x_on": phi_2_x_on_12,
    "phi_1_dcx_on": phi_1_dcx_on_12, "phi_2_dcx_on": phi_2_dcx_on_12, "M_12_on": M_12_on,
    "gamma": gamma
}


initial_state_used_choice = ["four_well", "after_CE_8_completed"]



# bookmark
initial_parameter_dict["phi_1_dcx"] = circuit_params_chosen["phi_1xdc"]
initial_parameter_dict["phi_2_dcx"] = circuit_params_chosen["phi_2xdc"]
initial_parameter_dict["phi_1_x"]   = circuit_params_chosen["phi_1x"]
initial_parameter_dict["phi_2_x"]   = circuit_params_chosen["phi_2x"]
initial_parameter_dict["M_12"]      = circuit_params_chosen["M_12"]

initial_parameter_dict["phi_1_dcx"] = 0
initial_parameter_dict["phi_2_dcx"] = 0
initial_parameter_dict["phi_1_x"]   = 0
initial_parameter_dict["phi_2_x"]   = 0
initial_parameter_dict["M_12"]      = 0

### modify the protocol

In [ ]:
CE_1 = {
        "phi_1_x": phi_1_x_on_12, "phi_2_x": phi_2_x_on_12, "M_12": M_12_on, \
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on_12, "name":"CE_1"
}

KE_extraction_protocol_for_CE_1 = {
        "phi_1_x": phi_1_x_on_12, "phi_2_x": 0.00, "M_12": 0.00, \
        "phi_1_dcx": phi_1_dcx_on_12, "phi_2_dcx": phi_2_dcx_on_12,\
        "name":"KE_extraction_protocol"
}


In [ ]:
protocol_type = "CE"

if protocol_type == "CE":
    mapping_dict = mapping_state_1_to_state_2_dict_CE
if protocol_type == "NAND":
    mapping_dict = mapping_state_1_to_state_2_dict_NAND


customized_protocol = {
    'phi_1_x': 0,
     'phi_2_x': 0,
     'M_12': 0,
     'phi_1_dcx': 0,
     'phi_2_dcx': 0.0,
     'name': 'sudden_jump'
}


protocol_list_NAND_protocol = [
    # forward
    create_CE_Protocol(10, customized_protocol),
]

protocol_list = protocol_list_NAND_protocol

# first initialization

### create initial state 

In [ ]:
%%capture
initial_state_array = ["saved_init_state/four_well_default_init_state_N_1000.npy", "saved_init_state/after_CE_8_completed_N_1000z.npy", "saved_init_state/just_before_KEP_N_1000.npy"]
initial_state_chosen = initial_state_array[0]


regenerate_init_state = False
# regenerate_init_state = True
if regenerate_init_state:
    initial_potential_protocol_list = [create_CE_Protocol(200*params['dt'], four_well)]
    init_params = params.copy()
    # friction_factor = 1
    # init_params['sim_params'] = [init_params['sim_params'][0] * friction_factor, init_params['sim_params'][1], init_params['sim_params'][2] * friction_factor]
    init_params['dt'] = 1/500
    init_state_saved = cfq_batch_sweep.create_initial_state(initial_parameter_dict, initial_potential_protocol_list, coupled_fq_pot, init_params)
else:
    init_state_saved = np.load(initial_state_chosen)

init_state_used = init_state_saved

# np.save("four_well_default_init_state_N_1000.npy", init_state_saved)

# particle_index = visualization.separate_by_state_2(init_state_saved[:, (0, 1), :])
# init_state_used = init_state_saved[particle_index["00"]]
# [1.3, 0.3, 0.3, 0.3, 1]

In [ ]:
plt.scatter(init_state_saved[..., 0, 0], init_state_saved[..., 1, 0])

### create cfqr object

In [ ]:
"""
# step 3: create the relevant storage protocol and computation protocol
"""
def modifiedFunction(t, result):
    return result


computation_protocol_parameter_dict = coupled_fq_protocol_library.customizedProtocol(initial_parameter_dict, \
                                                                    protocol_list)
storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict, modifiedFunction = None)

cfqr = cfq_runner.coupledFluxQubitRunner(potential = coupled_fq_pot, params = params, \
                                                storage_protocol= storage_protocol, \
                                                computation_protocol= comp_protocol, \
                                         protocol_list = protocol_list, \
                                        has_velocity=has_velocity)

protocol_time_array, protocol_time_index_array, protocol_all_time_array, protocol_time_all_index_array = cfqr.createProtocolTimeArray(protocol_list, params)
cfqr.initialize_sim()
# cfqr.set_sim_attributes(init_state=init_state_saved)
cfqr.set_sim_attributes(init_state=init_state_used)
print(f"mean KE = {np.mean(cfqr.system.get_kinetic_energy(init_state_used))}")

# actual simulations

In [ ]:
importlib.reload(cfq_batch_sweep)

# 0.5 * (x - 0.59)^2 + 0.5 * (y - 0.1)^2 + 2.3 * cos(x) + 2.3 * cos(y)
# step 5: perform simulations


manual_domain=[np.array([-5, -5]), np.array([5, 5])]

simResult = cfq_batch_sweep.simulateSingleCoupledFluxQubit(params, initial_parameter_dict, protocol_list, \
                        potential = coupled_fq_pot, potential_default_param = coupled_fq_default_param, mapping_state_1_to_state_2_dict = mapping_dict,         
                        initial_state = init_state_used, manual_domain = manual_domain, \
                        phi_1_dcx = phi_1_dcx,  phi_2_dcx = phi_2_dcx, measure_all_states=saveAllStates, \
                        has_velocity = has_velocity)
print(f"characteristic_time = {characteristic_time * 10e9:.3g}ns")

# sim Results

### fidelity

In [ ]:
import sys, importlib
# importlib.reload(sys.modules['edward_tools.jupyter_helper_function'])
from edward_tools.jupyter_helper_function import work_analysis, KE_analysis

# work done of each type

In [ ]:
work_done_of_each_type(simResult)

In [ ]:
tellMeImportantFactors(protocol_list)

###  single particle analysis

In [ ]:
import sys, importlib
importlib.reload(sys.modules['edward_tools.jupyter_helper_function'])
from edward_tools.jupyter_helper_function import work_analysis, KE_analysis, single_particle_analysis, fidelity_analysis, work_done_of_each_type
all_states = simResult['cfqr'].get_all_state()

particle_category = ["00"]
particle_index = 0

for _c in particle_category:
    fig, ax1 = plt.subplots(figsize=(8, 5))
    ax2 = ax1.twinx()
    index_choice = simResult['cfqr'].getIndexOfParticles()[_c]
    reduced_data = single_particle_analysis(simResult, params, protocol_list, plot_ax = ax1, 
                             particle_index=particle_index, particle_category= _c, 
                             time_range=None)
    reduced_protocal_time_array = reduced_data["reduced_protocal_time_array"]
    index_slice = reduced_data["index_slice"]

    # print(simResult['cfqr'].protocol_time_array)
    y_min, y_max = ax1.set_ylim()
    x_min, x_max = ax1.set_xlim()
    for _t in simResult['cfqr'].protocol_time_array:
        plt.vlines(x=_t, ymin = y_min, ymax = y_max, linestyles="--")


    all_states_00_i1 = all_states[index_choice][particle_index, ...]
    mean_phi_2 = all_states_00_i1[:, 1, 0]
    # mean_phi_1 = np.mean(all_states[index_choice][:, :, 0, 0], axis = 0)


    ax2.plot(reduced_protocal_time_array, mean_phi_2, alpha =0.8, color = "grey", label = f"avg phi_2 of {particle_category}")
    ax2.set_ylim(-4, 4)
    ax2.set_ylabel(r"$\varphi_{2}$")

    ax2.hlines(1.5, x_min, x_max, color = "red", linestyle = "--")
    ax2.hlines(0.5, x_min, x_max, color = "red", linestyle = "--")
    ax1.set_title(f"energy profile of {particle_category} particle (total time of protocol = {protocol_time_array[-1] * characteristic_time * 1e9:.3g} ns)")
    ax2.legend()
    plt.show()